# Pytorch Base Template

### MODULES

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

### CONFIGURATIONS

In [2]:
CFG = {
    "batch_size": 32,
    "lr": 0.001,
    "epochs": 10,
    "num_workers": 0
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### SAMPLE DATA

In [4]:
import numpy as np

samples = np.random.rand(1000, 5)
train_idx = np.arange(600)
val_idx = np.arange(600, 800)
test_idx = np.arange(800, 1000)
print(samples)
print(train_idx.shape)
print(val_idx.shape)
print(test_idx.shape)

[[0.83004821 0.66853652 0.87390994 0.25807092 0.76774931]
 [0.11565201 0.37706954 0.96101122 0.79003733 0.69426307]
 [0.7363972  0.59160882 0.91935905 0.87357674 0.51348941]
 ...
 [0.53159385 0.6996126  0.93819342 0.42920674 0.03848498]
 [0.71062043 0.94355812 0.4718376  0.72504445 0.10582   ]
 [0.91097073 0.31684282 0.98655006 0.63437644 0.0784837 ]]
(600,)
(200,)
(200,)


### DATASET

In [5]:
class Data(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx][:4]
        label = self.samples[idx][4:]

        sample, label = torch.tensor(sample).float(), torch.tensor(label).float()
        return sample, label


dataset = Data(samples)
example_input, example_target = dataset[0]
example_input, example_target

(tensor([0.8300, 0.6685, 0.8739, 0.2581]), tensor([0.7677]))

### DATALOADER

In [6]:
data_loader = {
    "train": DataLoader(dataset, sampler=train_idx, batch_size=CFG["batch_size"], shuffle=False, num_workers=CFG["num_workers"], pin_memory=True),
    "val": DataLoader(dataset, sampler=val_idx, batch_size=CFG["batch_size"], shuffle=False, num_workers=CFG["num_workers"], pin_memory=True),
    "test": DataLoader(dataset, sampler=test_idx, batch_size=CFG["batch_size"], shuffle=False, num_workers=CFG["num_workers"], pin_memory=True)  
}

### MODEL

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 4),
            nn.BatchNorm1d(4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.BatchNorm1d(4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        x = self.fc(x)
        return x


net = Net().to(device)

### TRAINING

In [8]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=CFG["lr"])

for epoch in range(CFG["epochs"]):
    history = {
            "train_loss": [],
            "val_loss": []
    }

    for phase in ["train", "val"]:
        if phase == "train":
            net.train()
        else:
            net.eval()

        for inputs, labels in data_loader[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()

            outputs = net(inputs)

            loss = criterion(outputs, labels)

            if phase == "train":
                loss.backward()
                optimizer.step()

            history[f"{phase}_loss"].append(loss.item())

    mean_train_loss = np.mean(history["train_loss"])
    mean_val_loss = np.mean(history["val_loss"])
    print(f"({epoch+1}) {phase.upper()} \t train_loss={mean_train_loss:.2f} \t val_loss={mean_val_loss:.2f}")

(1) VAL 	 train_loss=1.01 	 val_loss=0.74
(2) VAL 	 train_loss=0.84 	 val_loss=0.75
(3) VAL 	 train_loss=0.71 	 val_loss=0.70
(4) VAL 	 train_loss=0.61 	 val_loss=0.62
(5) VAL 	 train_loss=0.54 	 val_loss=0.55
(6) VAL 	 train_loss=0.47 	 val_loss=0.49
(7) VAL 	 train_loss=0.42 	 val_loss=0.44
(8) VAL 	 train_loss=0.38 	 val_loss=0.41
(9) VAL 	 train_loss=0.34 	 val_loss=0.37
(10) VAL 	 train_loss=0.31 	 val_loss=0.35
